*Final project for Quantitative Macroeconomic Theory at Washington University in St. Louis*
# Does the Type of Mortgage Rate Matter? Adjustable Rate and Fixed Rate
## Kam, Yu Fai (Geoffrey); Nemoto, Hiroyuki; and Tanaka, Masaki


## Step 0. Load modules and set parameters

### (A) Load modules

In [1]:
%matplotlib inline
from KNTModel import *
from KNTModel.tools import AR1_process
import numpy as np

### (B) Set parameters
#### parameters for the income ($z$) process


In [2]:
N_z     = 10    # number of grid points
rho_z   = 0.9   # AR1 coefficient
sig_z   = 0.008 # size of exogenous shock
Omega_z = 3     # range of grid

#### parameters for the aggregate factor ($x$) process

In [3]:
N_x     = 5 # number of grid points
rho_x   = 0.8 # AR1 coefficient
sig_x   = 0.05 # size of exogenous shock
Omega_x = 2 # range of grid

## Step 1. Discretize the exogenous processes
### (A) Discretize the income ($z$) process

In [4]:
# Generate the instance for the income process
lnz_process = AR1_process(rho = rho_z,
                          sig = sig_z,
                          varname = 'lnz')
# Discretize by Tauchen's method
lnz_process.discretize(method = 'Tauchen',
                       N = N_z,
                       Omega = Omega_z,
                       is_write_out_result = False)
# Get the grid and the transition probabilities
z_grid = np.exp(lnz_process.lnz_grid)
trans_prob_z = lnz_process.trans_mat

Discretizing the AR(1) process by Tauchen method...
Discretization done.


### (B) Discretize the aggregate factor ($x$) process

In [5]:
# Generate the instance for the aggregate factor process
x_process = AR1_process(rho = rho_x,
                        sig = sig_x,
                        varname = 'x')
# Discretize by Tauchen's method
x_process.discretize(method = 'Tauchen',
                     N = N_x,
                     Omega = Omega_x,
                     is_write_out_result = False)
# Get the grid and the transition probabilities
x_grid = x_process.x_grid + 1
trans_prob_x = x_process.trans_mat

Discretizing the AR(1) process by Tauchen method...
Discretization done.


## Step 2. Solve the adjustable-rate model

In [6]:
# Generate the instance for the adjustable-rate economy
ARM_Economy = ARM_FixedHousePrice(z_grid = z_grid,
                                  x_grid = x_grid,
                                  trans_prob_z = trans_prob_z,
                                  trans_prob_x = trans_prob_x,)

In [7]:
# Run the value iteration
ARM_Economy.value_func_iter()

Starting the value function iteration...
elapsed time = 0:02:38.975709



In [8]:
# Solve for the stationary distribution (under x = 1)
ARM_Economy.get_stationary_dist()

Starting to calculate the stationary distribution (under x = 1)...
elapsed time = 0:00:00.025329

Failed to obtain the stationary distribution. Try again with more max_iter.


## Step 3. Solve the (partially) fixed-rate model

In [9]:
# Generate the instance for the (partially) fixed-rate economy
FRM_Economy = FRM_FixedHousePrice(z_grid = z_grid,
                                  x_grid = x_grid,
                                  trans_prob_z = trans_prob_z,
                                  trans_prob_x = trans_prob_x,)

In [10]:
# Run the value iteration
FRM_Economy.value_func_iter()

Starting the value function iteration...
elapsed time = 0:02:36.699778



In [11]:
# Solve for the stationary distribution (under x = 1)
FRM_Economy.get_stationary_dist()

Starting to calculate the stationary distribution (under x = 1)...
elapsed time = 0:00:00.023502

Failed to obtain the stationary distribution. Try again with more max_iter.
